In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
import math

Case Data

In [ ]:
 pip install PYPOWER #installing PyPower

In [ ]:
from pypower.loadcase import loadcase
from pypower.makeYbus import makeYbus
from pypower import case30
 
case30

<module 'pypower.case30' from '/usr/local/lib/python3.7/dist-packages/pypower/case30.py'>

Loading the case (case30):

In [ ]:
from pypower.api import case30, ppoption, runpf, printpf

Accessing the Pmin and Pmax from the case and getting the slack bus:

In [ ]:
ppc = case30()
Gen = ppc["gen"]
Bus =[]
PMax= []
PMin=[]
i= 0 #iterator over every generator to get the Bus, Pmax , Pmin of the generator

while (i!= len(Gen)):
  Bus.append(ppc["gen"][i,0])
  PMax.append(ppc["gen"][i,8])
  PMin.append(ppc["gen"][i,9])
  i = i+1

Power= pd.DataFrame(columns=['Bus','Pmin','Pmax'])
Power['Bus']= Bus
Power['Pmin'] = PMin
Power['Pmax'] = PMax 
Power.set_index('Bus')
print(Power)
#finding the slack bus

slack = 0
j=0 #iterator over every bus row to check for the bus type
while(slack == 0):
  if(ppc["bus"][j,1]== 3):
    slack = ppc["bus"][j,0]
  j +=1
print(slack)
slack_adapted = int(slack) - 1  


    Bus  Pmin  Pmax
0   1.0   0.0  80.0
1   2.0   0.0  80.0
2  22.0   0.0  50.0
3  27.0   0.0  55.0
4  23.0   0.0  30.0
5  13.0   0.0  40.0
1.0


Getting the admittance matrix:

In [ ]:
from pypower.ext2int import ext2int
from pypower.makeBdc import makeBdc
from scipy.sparse import csr_matrix
ppc = ext2int(ppc)
BInputbaseMVA, BInputbus, BInputbranch = ppc['baseMVA'], ppc['bus'],  ppc['branch']

Bbus, Bf, Pbusinj, Pfinj = makeBdc(BInputbaseMVA, BInputbus, BInputbranch)


#print(Bbus.shape)
#print(Bbus)

Bbus = csr_matrix(Bbus).toarray() #convert the csr_matrix into an array 
#print(Bbus)


Importing the Pd, Pg and Va csv files:

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
PDdf = pd.read_csv('Pd (1).csv')
PGdf = pd.read_csv('Pg (1).csv')
VAdf = pd.read_csv('Va (1).csv')
#Successdf = pd.read_csv('Success.csv')
print(PGdf)
print(PDdf)
unNormalizedPD = PDdf
unNormalizedPG = PGdf

               1          2         22         27         23         13
0      45.048322  58.626653  22.415463  33.089500  16.038657  16.038657
1      45.024672  58.599625  22.407895  33.032786  16.019737  16.019737
2      44.589249  58.101998  22.268560  31.988605  15.671399  15.671399
3      45.067070  58.648080  22.421462  33.134460  16.053656  16.053656
4      44.932513  58.494301  22.378404  32.811782  15.946010  15.946010
...          ...        ...        ...        ...        ...        ...
29995  44.083112  57.523556  22.106596  30.774848  15.266489  15.266489
29996  44.957973  58.523398  22.386552  32.872838  15.966379  15.966379
29997  44.443008  57.934866  22.221762  31.637908  15.554406  15.554406
29998  44.880120  58.434422  22.361638  32.686138  15.904096  15.904096
29999  44.620667  58.137906  22.278614  32.063951  15.696534  15.696534

[30000 rows x 6 columns]
       1          2         3         4  ...  27  28        29         30
0      0  21.289671  2.378422  8.256

In [ ]:
#Getting PD_maxs and PD_mins for later use
PD_maxs = []
PD_mins = []
it =0
for column in PDdf.columns:
        if(it!= slack_adapted):
          PD_maxs.append(PDdf[column].max()) 
          PD_mins.append(PDdf[column].min())
        it = it +1
print(PD_maxs)

[23.8698528748171, 2.6399994635452204, 8.35988878594656, 0, 0, 25.079658171486503, 32.999990925458796, 0, 6.3799923116425195, 0, 12.319969075969901, 0, 6.81993108525692, 9.01999731096113, 3.84999808005221, 9.89987838917731, 3.5199881498812697, 10.449940855323801, 2.41997607914709, 19.2499972280387, 0, 3.51995391395599, 9.56993840391718, 0, 3.8498843797976696, 0, 0, 2.6399850504091003, 11.6599936215336]


To Per Unit Function

In [ ]:
def To_Per_Unit(P):
  P = P /  ppc['baseMVA']
  return P

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
def To_Per_Unit_Torch(P):
  N= np.zeros((1,1))
  N[0][0] =  ppc['baseMVA']
  N = torch.from_numpy(N)
  PerUnit = torch.divide(P,N)
  return PerUnit


Load sampling and pre-processing

In [ ]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(df_norm[column].max()!=0):
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm

PDdf= normalize(PDdf)
PDdf


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,0,0.405466,0.455030,0.931780,0,0,0.149518,0.903991,0,0.304941,0,0.842318,0,0.776178,0.376498,0.805031,0.397707,0.644534,0.502489,0.359326,0.568546,0,0.072207,0.562414,0,0.124925,0,0,0.958366,0.608373
1,0,0.037271,0.282871,0.122347,0,0,0.461993,0.986580,0,0.188665,0,0.212723,0,0.614888,0.365995,0.825043,0.662669,0.286497,0.714369,0.784793,0.885892,0,0.380749,0.260535,0,0.463682,0,0,0.881221,0.969447
2,0,0.278539,0.718506,0.305251,0,0,0.488232,0.548334,0,0.603014,0,0.127959,0,0.988418,0.589695,0.202439,0.727080,0.797246,0.351234,0.419713,0.132447,0,0.915423,0.876822,0,0.969615,0,0,0.306033,0.511792
3,0,0.650072,0.430505,0.062975,0,0,0.629213,0.835281,0,0.926680,0,0.223140,0,0.896679,0.324242,0.445413,0.585058,0.037944,0.826336,0.263218,0.045599,0,0.773981,0.909466,0,0.487792,0,0,0.152683,0.541676
4,0,0.734191,0.256183,0.897375,0,0,0.251493,0.546152,0,0.123368,0,0.568002,0,0.112751,0.908695,0.929829,0.327422,0.728380,0.770813,0.723061,0.369239,0,0.131879,0.907805,0,0.633857,0,0,0.032652,0.557441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0,0.000325,0.055119,0.872479,0,0,0.532802,0.057938,0,0.469931,0,0.010664,0,0.645856,0.215552,0.879228,0.731564,0.536613,0.611763,0.934253,0.570274,0,0.217498,0.329987,0,0.644005,0,0,0.055835,0.872862
29996,0,0.868287,0.324404,0.679173,0,0,0.731200,0.495611,0,0.301600,0,0.452070,0,0.393609,0.742605,0.203990,0.657958,0.079665,0.478537,0.470904,0.181898,0,0.731481,0.970508,0,0.681810,0,0,0.257034,0.081286
29997,0,0.465733,0.096915,0.771958,0,0,0.443194,0.329889,0,0.277207,0,0.442987,0,0.316981,0.426723,0.131839,0.661742,0.353211,0.466963,0.395928,0.856618,0,0.116118,0.656454,0,0.094506,0,0,0.095585,0.245515
29998,0,0.983727,0.639763,0.394774,0,0,0.792464,0.282496,0,0.052178,0,0.659800,0,0.269432,0.042118,0.999326,0.311937,0.461021,0.132817,0.091099,0.738241,0,0.537868,0.246211,0,0.993647,0,0,0.686499,0.582034


*Linear transformation and mapping dimension reduction*


In [ ]:
def getslackindex(PGdf):
  PGdf2=PGdf.copy()
  titles=list(PGdf2.columns)
  strslack=str(int(slack))
  temp=0
  t=0
  for i in range(len(titles)):
    if(titles[i]==strslack):
      temp1=titles[i]
      t=i
  return t

print(getslackindex(PGdf))

0


In [ ]:

def GetAlphafromPG(PGdf):
  Alpha = PGdf.copy()
  i = 0
  for column in PGdf: 
    Alpha[column] = Alpha[column] - Power.at[i, "Pmin"]
    Alpha[column] = Alpha[column]/ (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
    Pmax=Power.at[i,"Pmax"]  
    print(Pmax)
    i= i+1    
  return Alpha

def GetAlphafromPGws(PGdf):
  t=getslackindex(PGdf)
  Alpha = PGdf.copy()
  i = 0
  #find x get x
  
  for column in PGdf: 

    Alpha[column] = Alpha[column] - Power.at[i+1, "Pmin"]
    Alpha[column] = Alpha[column]/ (Power.at[i+1, "Pmax"]- Power.at[i+1, "Pmin"])
    Pmax=Power.at[i+1,"Pmax"]   
    print(Pmax)  
    i = i+1
      
      
  return Alpha


def GetPGfromAlpha(Alpha):
  i = 0
  PG = Alpha.copy()
  for column in PG:
    PG[column] = PG[column]* (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
    PG[column] = PG[column] + Power.at[i, "Pmin"]
    i= i+1
  return PG

def GetPGfromAlphaws(Alpha):
  t=getslackindex(PGdf)
  PG  = Alpha.copy()
  i = 0
  j = 0
  for column in PGdf:
    if(i == t):
      i = i + 1
    else:
      PG[j] = PG[j]* (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
      PG[j] = PG[j] + Power.at[i, "Pmin"]
      i= i+1
      j = j+1
  return PG

slackstring = str(int(slack))
PGdf_ws=PGdf.drop(labels= slackstring , axis=1,inplace=False)
print(GetAlphafromPG(PGdf))
print(GetAlphafromPGws(PGdf_ws))

80.0
80.0
50.0
55.0
30.0
40.0
              1         2        22        27        23        13
0      0.563104  0.732833  0.448309  0.601627  0.534622  0.400966
1      0.562808  0.732495  0.448158  0.600596  0.533991  0.400493
2      0.557366  0.726275  0.445371  0.581611  0.522380  0.391785
3      0.563338  0.733101  0.448429  0.602445  0.535122  0.401341
4      0.561656  0.731179  0.447568  0.596578  0.531534  0.398650
...         ...       ...       ...       ...       ...       ...
29995  0.551039  0.719044  0.442132  0.559543  0.508883  0.381662
29996  0.561975  0.731542  0.447731  0.597688  0.532213  0.399159
29997  0.555538  0.724186  0.444435  0.575235  0.518480  0.388860
29998  0.561001  0.730430  0.447233  0.594293  0.530137  0.397602
29999  0.557758  0.726724  0.445572  0.582981  0.523218  0.392413

[30000 rows x 6 columns]
80.0
50.0
55.0
30.0
40.0
              2        22        27        23        13
0      0.732833  0.448309  0.601627  0.534622  0.400966
1      0.732495

Removing the slack bus from  PD , PG:

In [ ]:
slackstring = str(int(slack))
PDdf_ws=PDdf.drop(labels= slackstring , axis=1,inplace=False)
print(PDdf_ws) #PD without slack
PGdf_ws=PGdf.drop(labels= slackstring , axis=1,inplace=False)
print(PGdf_ws) #PG without slack
# PG NOT ALPHAC HERE CARFUL
unNormalizedPD_ws = unNormalizedPD.drop(labels= slackstring , axis=1,inplace=False)
#unNormalizedPG_ws = unNormalizedPG.drop(labels= slackstring , axis=1,inplace=False)





              2         3         4  5  6  ...        26  27  28        29        30
0      0.405466  0.455030  0.931780  0  0  ...  0.124925   0   0  0.958366  0.608373
1      0.037271  0.282871  0.122347  0  0  ...  0.463682   0   0  0.881221  0.969447
2      0.278539  0.718506  0.305251  0  0  ...  0.969615   0   0  0.306033  0.511792
3      0.650072  0.430505  0.062975  0  0  ...  0.487792   0   0  0.152683  0.541676
4      0.734191  0.256183  0.897375  0  0  ...  0.633857   0   0  0.032652  0.557441
...         ...       ...       ... .. ..  ...       ...  ..  ..       ...       ...
29995  0.000325  0.055119  0.872479  0  0  ...  0.644005   0   0  0.055835  0.872862
29996  0.868287  0.324404  0.679173  0  0  ...  0.681810   0   0  0.257034  0.081286
29997  0.465733  0.096915  0.771958  0  0  ...  0.094506   0   0  0.095585  0.245515
29998  0.983727  0.639763  0.394774  0  0  ...  0.993647   0   0  0.686499  0.582034
29999  0.037689  0.446949  0.995120  0  0  ...  0.415951   0   0 

Removing the slack bus from admittance matrix:

In [ ]:
slack_adapted = int(slack) - 1  
B_tilda = np.delete(Bbus, slack_adapted,0) 
B_tilda = np.delete(B_tilda, slack_adapted,1)

Getting the Phase Angle (without the slack bus), for uploaded values

In [ ]:
def Get_B_Matrix_Inverse():
  BInputbaseMVA, BInputbus, BInputbranch = ppc['baseMVA'], ppc['bus'],  ppc['branch']
  Bbus, Bf, Pbusinj, Pfinj = makeBdc(BInputbaseMVA, BInputbus, BInputbranch)
  Bbus = csr_matrix(Bbus).toarray()
  slack_adapted = int(slack) - 1   
  B_tilda = np.delete(Bbus, slack_adapted,0) 
  B_tilda = np.delete(B_tilda, slack_adapted,1)

  #First need to invert the admittance matrix found previously
  B_inverse = np.linalg.inv(B_tilda) 
  return B_inverse 

In [ ]:
def Get_Phase_Angles(PG, unNormalizedPD_ws):
  
  BInputbaseMVA, BInputbus, BInputbranch = ppc['baseMVA'], ppc['bus'],  ppc['branch']
  Bbus, Bf, Pbusinj, Pfinj = makeBdc(BInputbaseMVA, BInputbus, BInputbranch)
  Bbus = csr_matrix(Bbus).toarray()
  slack_adapted = int(slack) - 1   
  B_tilda = np.delete(Bbus, slack_adapted,0) 
  B_tilda = np.delete(B_tilda, slack_adapted,1)

  #First need to invert the admittance matrix found previously
  B_inverse = np.linalg.inv(B_tilda) 
  #print("B inverse:")
  #print(B_inverse)
  PDtemp = unNormalizedPD_ws.copy() # for this formula we need the unnormalized PD
  #preparing the PG matrix
  PG_full = pd.DataFrame(np.zeros(PDtemp.shape), columns = PDtemp.columns.tolist()) 
  PG_gen_list = PG.columns.tolist()
  for col in PG_full.columns:
    if col in PG_gen_list:
      PG_full[col].values[:] = PG[col].values[:] 

  #Preparing PD and PG
  PDtemp= To_Per_Unit(PDtemp) #turning PDs into per unit value
  PG_full= To_Per_Unit(PG_full) #turning the P generated values
  PDtemp = np.transpose(pd.DataFrame.to_numpy(PDtemp)) #transposing to have the buses as the rows and the cases as the columns
  PG_full= np.transpose(pd.DataFrame.to_numpy(PG_full)) #transposing to have the buses as the rows and the cases as the columns

  #getting the difference between PG and PD
  Difference = PG_full - PDtemp 
  #getting the angles in degrees
  Angles =(180/math.pi)*np.matmul(B_inverse,Difference)

  return Angles
Angles0 = Get_Phase_Angles(PGdf_ws, unNormalizedPD_ws)
print(Angles0)

[[-0.79461668 -0.7897442  -0.78675782 ... -0.78937454 -0.81253821
  -0.77802032]
 [-2.38776338 -2.40061833 -2.36267415 ... -2.33846783 -2.31270106
  -2.39376324]
 [-2.835941   -2.85339607 -2.80267147 ... -2.78020687 -2.74304393
  -2.84329288]
 ...
 [-3.22935222 -3.30811868 -3.21508947 ... -3.1572901  -3.12040524
  -3.22810976]
 [-2.89907019 -3.18240777 -2.98744725 ... -2.76193018 -2.81297204
  -2.91485982]
 [-3.84575916 -4.2123786  -3.9356549  ... -3.65817364 -3.76339781
  -3.84585096]]


Getting the Phase Angle (with Slack)

In [ ]:
def getAnglesWithSlack(Angles):
  #We have to add 0s for the slack's phase angle 
  numberofrowsA , numberofcolumnsA= Angles.shape 
  numberofcolumnsA = numberofcolumnsA
  slackangles = [[0]*numberofcolumnsA]
  #print(slackangles)
  #np.append(Angles,slackangles,0)
  Angles = np.insert(Angles, int(slack_adapted) , slackangles , 0)
  return Angles
#Angles0 = getAnglesWithSlack(Angles0)
#print(Angles0)
#print(getAnglesWithSlack(Angles))


Loss Functions:
Getting LPG:


In [ ]:
#Penalty Function
def penalty(x):
  p = np.square(x) - 1
  return p

#A Matrix Function
def createAmatrix():
  n1, m1 = ppc['bus'].shape
  n2, m2 = ppc['branch'].shape
  A = np.empty((n2,n1))
  A[:] = 0
  Adf = pd.DataFrame(A)
  for row in range(len(Adf)):
    Adf[ppc['branch'][row][0]][row]= 1/ ((ppc['branch'][row][3]) * (To_Per_Unit(ppc['branch'][row][5]))) #ai = 1/ xij Pijmax and ask prof jabr if the power in pu or angles in degrees or radiant 
    Adf[ppc['branch'][row][1]][row]= -1 / ((ppc['branch'][row][3]) * (To_Per_Unit(ppc['branch'][row][5]))) #aj = - 1/ xij Pijmax 
  A = pd.DataFrame(Adf).to_numpy(dtype= 'float')
  return A
print(createAmatrix())

[[ 12.82051282 -12.82051282   0.         ...   0.           0.
    0.        ]
 [  4.048583     0.          -4.048583   ...   0.           0.
    0.        ]
 [  0.           9.04977376   0.         ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   0.          13.88888889
  -13.88888889]
 [  0.           0.           0.         ... -15.625        0.
    0.        ]
 [  0.           0.           0.         ... -52.08333333   0.
    0.        ]]


Preparing the data:

In [ ]:
#drop the  0 values and you are left with 20 colums --> input to 1st layer like in the paper 
import numpy as np
import pandas as pd

PDdf_withoutzeros= PDdf_ws.loc[:, (PDdf_ws != 0).any(axis=0)]
PDdf_withoutzeros

Alpha = GetAlphafromPGws(PGdf_ws)
Alpha


80.0
50.0
55.0
30.0
40.0


,2,22,27,23,13
0,0.732833,0.448309,0.601627,0.534622,0.400966
1,0.732495,0.448158,0.600596,0.533991,0.400493
2,0.726275,0.445371,0.581611,0.522380,0.391785
3,0.733101,0.448429,0.602445,0.535122,0.401341
4,0.731179,0.447568,0.596578,0.531534,0.398650
...,...,...,...,...,...
29995,0.719044,0.442132,0.559543,0.508883,0.381662
29996,0.731542,0.447731,0.597688,0.532213,0.399159
29997,0.724186,0.444435,0.575235,0.518480,0.388860
29998,0.730430,0.447233,0.594293,0.530137,0.397602


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as functional
PD_train, PD_test, Alpha_train, Alpha_test = train_test_split(PDdf_withoutzeros, Alpha, test_size=0.2, random_state=42)
PD_train 

,2,3,4,7,8,10,12,14,15,16,17,18,19,20,21,23,24,26,29,30
21753,0.326233,0.261585,0.307766,0.156268,0.875702,0.703217,0.048082,0.107906,0.855895,0.318211,0.996487,0.390526,0.559748,0.624869,0.399576,0.174140,0.741631,0.053121,0.258273,0.666075
251,0.090263,0.874544,0.049550,0.600799,0.598558,0.449021,0.719142,0.706332,0.468379,0.158967,0.520719,0.948684,0.615455,0.105746,0.324349,0.687170,0.776208,0.256462,0.685704,0.978176
22941,0.829108,0.703122,0.793971,0.534843,0.552323,0.742004,0.116631,0.208368,0.117430,0.771318,0.809594,0.671080,0.091493,0.227012,0.843115,0.078803,0.714859,0.691393,0.858181,0.294862
618,0.638255,0.899094,0.060494,0.578574,0.776060,0.274256,0.049454,0.629864,0.476406,0.614210,0.044950,0.426453,0.067925,0.350169,0.949851,0.680203,0.169360,0.459577,0.033467,0.151890
17090,0.959654,0.098523,0.181575,0.717083,0.631474,0.544851,0.133724,0.403053,0.585879,0.954691,0.409456,0.572179,0.426808,0.836225,0.418924,0.397776,0.404316,0.406958,0.596566,0.925438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,0.721061,0.043455,0.330621,0.251876,0.118936,0.340421,0.362696,0.276083,0.434513,0.472894,0.953602,0.591843,0.211443,0.335261,0.978438,0.235401,0.340363,0.975712,0.374618,0.898364
5390,0.088903,0.304913,0.882608,0.824978,0.328853,0.151823,0.383440,0.651274,0.970572,0.176247,0.847587,0.340052,0.238121,0.033508,0.814657,0.467412,0.262362,0.109288,0.186941,0.749851
860,0.475176,0.780210,0.996192,0.494499,0.319835,0.360475,0.853765,0.019574,0.617945,0.637515,0.051853,0.426979,0.189149,0.357077,0.663095,0.123859,0.651236,0.379087,0.407397,0.498528
15795,0.081447,0.684850,0.060551,0.161134,0.615903,0.128263,0.063234,0.489304,0.546971,0.577888,0.633145,0.974898,0.072269,0.658342,0.979885,0.376113,0.873052,0.695869,0.981832,0.674214


Initialize the network

Training data to tensors:

In [ ]:
#INPUTS 
#Convert to Array 
inputs = PD_train.values
inputs = inputs.astype ('float32') #emphasizing on the type

#Convert into tensors
inputs = torch.from_numpy(inputs)
print(inputs)

#TARGETS
#Convert to Array 
Alpha_target = Alpha_train.values
Alpha_target = Alpha_target.astype ('float32') #emphasizing on the type

#Convert into tensors
Alpha_target = torch.from_numpy(Alpha_target)
print(Alpha_target)



tensor([[0.3262, 0.2616, 0.3078,  ..., 0.0531, 0.2583, 0.6661],
        [0.0903, 0.8745, 0.0495,  ..., 0.2565, 0.6857, 0.9782],
        [0.8291, 0.7031, 0.7940,  ..., 0.6914, 0.8582, 0.2949],
        ...,
        [0.4752, 0.7802, 0.9962,  ..., 0.3791, 0.4074, 0.4985],
        [0.0814, 0.6848, 0.0606,  ..., 0.6959, 0.9818, 0.6742],
        [0.5444, 0.1823, 0.8109,  ..., 0.7490, 0.2140, 0.2211]])
tensor([[0.7272, 0.4458, 0.5843, 0.5240, 0.3930],
        [0.7293, 0.4467, 0.5908, 0.5280, 0.3960],
        [0.7327, 0.4483, 0.6013, 0.5344, 0.4008],
        ...,
        [0.7260, 0.4452, 0.5807, 0.5218, 0.3914],
        [0.7244, 0.4445, 0.5759, 0.5189, 0.3892],
        [0.7242, 0.4444, 0.5753, 0.5185, 0.3889]])


Testing data to tensors:

In [ ]:
#INPUTS 
#Convert to Array 
inputs_test = PD_test.values
inputs_test = inputs_test.astype ('float32') #emphasizing on the type

#Convert into tensors
inputs_test = torch.from_numpy(inputs_test)
print(inputs_test)

#TARGETS
#Convert to Array 
Alpha_target_test = Alpha_test.values
Alpha_target_test = Alpha_target_test.astype ('float32') #emphasizing on the type

#Convert into tensors
Alpha_target_test = torch.from_numpy(Alpha_target_test)
print(Alpha_target_test)

tensor([[0.1916, 0.1734, 0.5100,  ..., 0.0992, 0.8930, 0.1791],
        [0.5186, 0.7394, 0.5469,  ..., 0.0855, 0.8300, 0.1611],
        [0.1492, 0.7463, 0.3598,  ..., 0.8811, 0.1138, 0.4091],
        ...,
        [0.6764, 0.4581, 0.2151,  ..., 0.0855, 0.9290, 0.0580],
        [0.7664, 0.3700, 0.2338,  ..., 0.3806, 0.7530, 0.5826],
        [0.1603, 0.3225, 0.2677,  ..., 0.4790, 0.7139, 0.2999]])
tensor([[0.7324, 0.4481, 0.6004, 0.5339, 0.4004],
        [0.7153, 0.4404, 0.5480, 0.5018, 0.3764],
        [0.7157, 0.4406, 0.5494, 0.5027, 0.3770],
        ...,
        [0.7324, 0.4481, 0.6003, 0.5338, 0.4004],
        [0.7235, 0.4441, 0.5732, 0.5172, 0.3879],
        [0.7207, 0.4429, 0.5645, 0.5119, 0.3839]])


In [ ]:
class DeepOPF(nn.Module):

  def __init__(self,inputSize,outputSize,hiddenSize1,hiddenSize2):
    super(DeepOPF, self).__init__()
    self.function1 = nn.Linear(inputSize,hiddenSize1)
    self.function2 = nn.Linear(hiddenSize1,hiddenSize2)
    self.function3 = nn.Linear(hiddenSize2,outputSize)


  def forward(self,x):
    x = torch.relu(self.function1(x))
    x = torch.relu(self.function2(x))
    x = torch.sigmoid(self.function3(x))
    return x

In [ ]:
  inputSize = 20
  outputSize = 5
  hiddenSize1 = 16
  hiddenSize2 = 16
  batch_size = 64
  Deep_OPF = DeepOPF(inputSize,outputSize,hiddenSize1,hiddenSize2)
  learning_rate = 1e-3
  n_epochs = 235
  emp = []
  for i in range(len(PD_train)):
    emp.append([inputs[i],Alpha_target[i]])
  

In [ ]:
 DataLoad = torch.utils.data.DataLoader(dataset = emp, batch_size = 64, shuffle = True)
 optimizer = torch.optim.Adam(Deep_OPF.parameters(), lr= learning_rate)

In [ ]:
A= createAmatrix() #getting A matrix 
Ones = np.zeros(shape=(np.shape(A)[0],1))
A = A.astype(np.float32)
A = torch.from_numpy(A) # A matrix as a tensor

#getting ones for loss 2
for x in range(len(Ones)):
  Ones[x] =1
Ones = torch.from_numpy(Ones)

#getting the difference for unnormalization
PDdiff = []
PDdifftorch = []
it = 0
for column in PD_maxs:
  PDdifftorch.append(PD_maxs[it]  - PD_mins[it] )
  it = it +1
print(PDdifftorch)
PDdiff = PDdifftorch.copy()
PDdifftorch = torch.from_numpy(np.asarray(PDdifftorch, dtype = 'float64')) 
print(PDdifftorch)

# getting min for unnormalization 

PDmintorch = torch.from_numpy(np.asarray(PD_mins, dtype = 'float64'))  

 #getting PG diff for getting PG from alphas

PGdifftorch = []
PGmin  = []
it=0
PGmax =  Power[:]["Pmax"]
for column in PGmax:
    if (it != slack_adapted):
      PGdifftorch.append(Power.at[it, "Pmax"]- Power.at[it, "Pmin"])
      PGmin.append(Power.at[it,"Pmin"])
    it= it+1
PGdifftorch = torch.from_numpy(np.asarray(PGdifftorch, dtype = 'float64'))

 #getting PG min for getting PG from alphas

PGmintorch = torch.from_numpy(np.asarray(PGmin, dtype = 'float64'))

#getting the B inverse for later use

B_inverse = Get_B_Matrix_Inverse()
B_inverse = torch.from_numpy(B_inverse)

[4.339840785442899, 0.4799853111504304, 1.5198377606236297, 0, 0, 4.559622075100403, 5.999952024951995, 0, 1.1598520370930094, 0, 2.239819785616202, 0, 1.23991983532175, 1.639938966084161, 0.6999792328551901, 1.7998777609062113, 0.6399268372083795, 1.899899531216521, 0.43997592847423994, 3.4998491520126027, 0, 0.6399360376218701, 1.7399046962123297, 0, 0.6998228629525398, 0, 0, 0.47998289811851036, 2.119934116978561]
tensor([4.3398, 0.4800, 1.5198, 0.0000, 0.0000, 4.5596, 6.0000, 0.0000, 1.1599,
        0.0000, 2.2398, 0.0000, 1.2399, 1.6399, 0.7000, 1.7999, 0.6399, 1.8999,
        0.4400, 3.4998, 0.0000, 0.6399, 1.7399, 0.0000, 0.6998, 0.0000, 0.0000,
        0.4800, 2.1199], dtype=torch.float64)


In [ ]:
from time import time
t = time()
for epochs in range(n_epochs):
  iteration = 0
  Loss_per_batch = []
  for i,(inputs,Alpha_target) in enumerate(DataLoad):
    optimizer.zero_grad()
    output_pred = Deep_OPF(inputs)
    loss1 = torch.nn.MSELoss(size_average=None, reduce=None, reduction = 'mean') #
    Loss1_input = loss1(output_pred ,Alpha_target) 
    #getting PG from output alpha
    PGOut0 = torch.mul(output_pred, PGdifftorch)
    PGOut0 = torch.add(PGOut0, PGmintorch)
    #getting PDs from input tensors 
    PDIn = torch.zeros(1,len(unNormalizedPD_ws.columns.to_list()))
    PGout = torch.zeros(1,len(unNormalizedPD_ws.columns.to_list()))
    columnOfInput = 0
    PDbuscolumn =0
    #adding the 0s
    for i in unNormalizedPD_ws:
      if (unNormalizedPD_ws[i].max() != 0):
        PDIn[0,PDbuscolumn]= inputs[0,columnOfInput]
        columnOfInput = columnOfInput + 1
      PDbuscolumn =   PDbuscolumn + 1
        #unNormalizing PD
    PDIn = torch.mul(PDIn, PDdifftorch)
    PDIn = torch.add(PDIn, PDmintorch)
    columnOfOutput = 0
    PGbuscolumn =0
    #getting theta tilda
    PG_gen_list = PGdf_ws.columns.tolist()
    for col in unNormalizedPD_ws.columns:
      if col in PG_gen_list:
        PGout[0,PGbuscolumn] = PGOut0[0,columnOfOutput]
        columnOfOutput = columnOfOutput + 1
      PGbuscolumn =   PGbuscolumn + 1
    # preparing PD and PG in per unit
    PDInFinal =  To_Per_Unit_Torch(PDIn)
    PGOutFinal = To_Per_Unit_Torch(PGout)
    PDInFinal = torch.transpose(PDInFinal, 0, 1)
    PGOutFinal = torch.transpose(PGOutFinal, 0, 1)
    Difference = torch.subtract(PGOutFinal,PDInFinal) #getting the difference between PG and PD
    Angles =torch.matmul(B_inverse,Difference)   #getting the angles in randiants / multiply by 180/pi for randiants
    length_withslack = 1+len(unNormalizedPD_ws.columns.to_list())
    Angles_hat = torch.zeros([length_withslack,1] )
    #getting angles with slack
    it=0
    for i in range(length_withslack):
      if i != slack_adapted:
      
        Angles_hat[i,0] = Angles[it,0]
        it = it +1
    
    #calculating loss 2
    Loss2_input = torch.matmul(A,Angles_hat)    
    Loss2_input = torch.square(Loss2_input)
    Loss2_input = torch.subtract(Loss2_input,Ones)
    Loss2_input = torch.mean(Loss2_input)
    Total_Loss = Loss1_input + Loss2_input* 0.00001
    Total_Loss.backward()#
    optimizer.step()#
   
  print(epochs, Loss1_input.item())
  

print(f'Time to run: {(time() - t)} seconds')

    

0 7.54303764551878e-05
1 2.006040813284926e-05
2 1.0401129202364245e-06
3 4.653347787098028e-07
4 2.501290055079153e-07
5 1.8621315689415496e-07
6 1.2490438905388146e-07
7 7.11955507881612e-08
8 7.606234220247643e-08
9 3.9057503897765855e-08
10 2.7791497103635265e-08
11 2.261485221310977e-08
12 2.2228585194739026e-08
13 3.518406543889796e-08
14 1.8962843739700475e-08
15 5.5488051486918266e-08
16 2.0522842092418614e-08
17 3.441164508899419e-08
18 4.102790995830219e-08
19 2.9053200734097118e-08
20 2.6689495058462853e-08
21 2.3242744617846256e-08
22 2.516742014790907e-08
23 4.11189233773257e-08
24 2.0522925581190066e-08
25 3.3962795242814536e-08
26 2.4394628539425867e-08
27 3.4254831859925616e-08
28 1.8030167581173373e-08
29 2.352147454587339e-08
30 1.8566103321404626e-08
31 1.469568822187739e-08
32 3.980840901363081e-08
33 2.4632882400510425e-08
34 1.6393297386230188e-08
35 2.0873795136822082e-08
36 4.658906505028426e-08
37 2.212555649805381e-08
38 1.4934423475665426e-08
39 6.79576714901

In [ ]:
from time import time
from numpy import vstack
import numpy as np
import math

from sklearn.metrics import accuracy_score
test_data = []
for i in range(len(PD_test)):
    test_data.append([inputs_test[i],Alpha_target_test[i]])
TestLoad = torch.utils.data.DataLoader(dataset = test_data, batch_size = 64, shuffle = True)
predictions, actuals = list(), list()
t = time()
for i, (inputs_test, Alpha_target_test) in enumerate(test_data):
    # evaluate the model on the test set
    yhat = Deep_OPF(inputs_test)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    actual = Alpha_target_test.numpy()
    predictions.append(yhat)
    actuals.append(actual)
print(f'Time to run: {(time() - t)} seconds')


Time to run: 0.5357749462127686 seconds


code for saving the model 

In [ ]:
print(actuals[2],predictions[2])
print(actuals[1000],predictions[1000])


[0.71573234 0.4406481  0.5494338  0.5027004  0.3770253 ] [0.71545273 0.44080827 0.54930454 0.50268567 0.3771508 ]
[0.71095246 0.43850672 0.5348452  0.49377796 0.37033346] [0.71060616 0.43867287 0.53454655 0.49377975 0.37051332]


Average of Root Mean Squared error of 15000 testing simulations

In [ ]:
from sklearn.metrics import mean_squared_error
n=[]
actuals = np.array(actuals)
predictions = np.array(predictions)
for i in range(len(actuals[0])):
  s=mean_squared_error(actuals[i],predictions[i],squared=False)
  n.append(s)
n=np.array(n)
print(n)
print((n.sum()/len(n)))


[0.0002013  0.00019271 0.00016523 0.00020408 0.00019208]
0.0001910800114274025


Average R2 SCORE 

In [ ]:
from sklearn.metrics import r2_score
n=[]
actuals = np.array(actuals)
predictions = np.array(predictions)
for i in range(len(actuals[0])):
  c=r2_score(actuals[i], predictions[i])
  n.append(c)
n=np.array(n) 
print(n)
print((n.sum()/len(n)))

[0.99999705 0.99999719 0.99999794 0.99999695 0.99999727]
0.9999972803690106


Saving the model

In [ ]:
PATH="model.pth"
torch.save(Deep_OPF.state_dict(), PATH)

Gettig the real Pg

In [ ]:
actuals_df = pd.DataFrame(actuals)
PG_actual_ws = GetPGfromAlphaws(actuals_df)
print(PG_actual_ws)
predictions_df = pd.DataFrame(predictions)
PG_predicted_ws = GetPGfromAlphaws(predictions_df)
print(PG_predicted_ws)
PG_predicted_ws.to_csv(r'PG_predictions_ws.csv',index=False)

              0          1          2          3          4
0     58.594700  22.406515  33.022453  16.016291  16.016291
1     57.221176  22.021929  30.140352  15.054821  15.054822
2     57.258587  22.032406  30.218861  15.081012  15.081013
3     58.499023  22.379726  32.821686  15.949316  15.949315
4     58.190990  22.293476  32.175339  15.733693  15.733692
...         ...        ...        ...        ...        ...
5995  57.655075  22.143421  31.050819  15.358554  15.358553
5996  58.948406  22.505552  33.764637  16.263884  16.263884
5997  58.591759  22.405693  33.016281  16.014231  16.014231
5998  57.880474  22.206533  31.523777  15.516333  15.516331
5999  57.652428  22.142679  31.045254  15.356696  15.356697

[6000 rows x 5 columns]
              0          1          2          3          4
0     58.567200  22.415165  33.013878  16.012819  16.021479
1     57.194839  22.030704  30.130928  15.053440  15.059727
2     57.236217  22.040413  30.211750  15.080570  15.086032
3     58.471851

Getting the phase angles

In [ ]:
R = len(PD_test.index)
C= len(unNormalizedPD_ws.columns)
PD_test_unNormalized = np.zeros((R,C))
PD_test = PD_test.to_numpy()
columnOfInput = 0
PDbuscolumn = 0
for i in unNormalizedPD_ws:
      if (unNormalizedPD_ws[i].max() != 0):
        for row in range(R):
          PD_test_unNormalized[row,PDbuscolumn]= PD_test[row,columnOfInput]
          PD_test_unNormalized[row,PDbuscolumn] = PD_test_unNormalized[row,PDbuscolumn] * PDdiff[PDbuscolumn]
          PD_test_unNormalized[row,PDbuscolumn] = PD_test_unNormalized[row,PDbuscolumn] + PD_mins[PDbuscolumn]
        columnOfInput = columnOfInput + 1
      PDbuscolumn =   PDbuscolumn + 1

PD_test_unNormalized =  pd.DataFrame(PD_test_unNormalized)
print(PD_test)
print(PD_test_unNormalized)
print(PG_predicted_ws)

[[0.1915565  0.17340502 0.51001443 ... 0.09920891 0.89299053 0.17912129]
 [0.51855232 0.73938317 0.54687602 ... 0.08554792 0.8299625  0.16110764]
 [0.14916682 0.74633539 0.35976073 ... 0.88112778 0.11378361 0.40906251]
 ...
 [0.67644487 0.45811891 0.21505287 ... 0.08550523 0.92898504 0.05804302]
 [0.76637364 0.36997239 0.23375148 ... 0.38056996 0.75295636 0.58258011]
 [0.16033363 0.32254791 0.26773079 ... 0.47901208 0.71392432 0.29990037]]
             0         1         2    3   ...   25   26        27         28
0     20.361337  2.243246  7.615190  0.0  ...  0.0  0.0  2.588622   9.919785
1     21.780447  2.514907  7.671214  0.0  ...  0.0  0.0  2.558370   9.881597
2     20.177372  2.518244  7.386829  0.0  ...  0.0  0.0  2.214616  10.407245
3     22.339288  2.412296  7.323495  0.0  ...  0.0  0.0  2.276543   9.730261
4     19.569104  2.475544  8.088795  0.0  ...  0.0  0.0  2.286665   9.752798
...         ...       ...       ...  ...  ...  ...  ...       ...        ...
5995  23.433518  

In [ ]:

angles_actual=Get_Phase_Angles(PG_actual_ws, PD_test_unNormalized)
angles_actual = pd.DataFrame(angles_actual)
angles_actual = angles_actual.T 
print(angles_actual)
angles_actual.to_csv(r'Angles_actuals_ws.csv',index=False)


            0         1         2   ...        26         27         28
0    -0.880890 -2.111521 -3.018160  ... -4.760084 -10.574489 -11.427726
1    -0.861135 -2.043244 -2.920469  ... -4.547473 -10.273555 -11.125001
2    -0.853088 -2.072290 -2.955793  ... -4.614216 -10.480345 -11.412431
3    -0.893300 -2.063110 -2.955069  ... -4.683973 -10.321971 -11.178252
4    -0.861097 -2.135743 -3.039567  ... -4.737090 -10.542430 -11.400337
...        ...       ...       ...  ...       ...        ...        ...
5995 -0.881449 -2.020248 -2.898258  ... -4.535687 -10.159698 -11.000703
5996 -0.899359 -2.086727 -2.987944  ... -4.731503 -10.751870 -11.715512
5997 -0.892630 -2.074063 -2.969665  ... -4.696910 -10.423641 -11.248592
5998 -0.883365 -2.035651 -2.919572  ... -4.634814 -10.592125 -11.540323
5999 -0.862940 -2.078606 -2.970628  ... -4.677217 -10.485421 -11.371934

[6000 rows x 29 columns]


In [ ]:
#@title 

angles_predicted=Get_Phase_Angles(PG_predicted_ws, PD_test_unNormalized)
angles_predicted = pd.DataFrame(angles_predicted)
angles_predicted = angles_predicted.T 
print(angles_predicted)
angles_predicted.to_csv(r'Angles_actuals_ws.csv',index=False)


            0         1         2   ...        26         27         28
0    -0.881684 -2.111804 -3.018701  ... -4.760627 -10.575032 -11.428269
1    -0.861864 -2.043490 -2.920968  ... -4.547956 -10.274039 -11.125485
2    -0.853640 -2.072381 -2.956087  ... -4.614492 -10.480622 -11.412709
3    -0.894024 -2.063313 -2.955507  ... -4.684382 -10.322383 -11.178665
4    -0.861770 -2.135858 -3.039900  ... -4.737419 -10.542760 -11.400667
...        ...       ...       ...  ...       ...        ...        ...
5995 -0.882040 -2.020344 -2.898563  ... -4.536018 -10.160027 -11.001032
5996 -0.899807 -2.086400 -2.987789  ... -4.731382 -10.751744 -11.715387
5997 -0.893429 -2.074286 -2.970145  ... -4.697387 -10.424119 -11.249070
5998 -0.883915 -2.035812 -2.919925  ... -4.635184 -10.592493 -11.540691
5999 -0.863465 -2.078692 -2.970909  ... -4.677532 -10.485731 -11.372244

[6000 rows x 29 columns]


Average of Root Mean Squared error of 15000 Angles and PG Testing Results

In [ ]:
from sklearn.metrics import mean_squared_error
n1=[]
n2=[]
actualAngles = np.array(angles_actual)
predictionAngles = np.array(angles_predicted)
actualPG = np.array(PG_actual_ws)
predictionPG = np.array(PG_predicted_ws)
print(len(actualPG[0]))
for i in range(len(actualPG[0])):
  s1=mean_squared_error(actualPG[i],predictionPG[i],squared=False)
  n1.append(s1)
n1=np.array(n1)
print("For PG:")
print(n1)
print(n1.sum()/len(n1))
for i in range(len(actualAngles[0])):
  s2=mean_squared_error(actualAngles[i],predictionAngles[i],squared=False)
  n2.append(s2)
n2=np.array(n2)
print("For Angles:")
print(n2)
print(n2.sum()/len(n2))

5
For PG:
[0.01373741 0.01330724 0.01131769 0.01373816 0.01313978]
0.013048055768013
For Angles:
[5.65524171e-04 5.02531261e-04 2.99188712e-04 4.44173326e-04
 3.63619739e-04 4.17248186e-04 1.33981508e-04 6.42951981e-04
 4.83971768e-04 2.86836594e-04 2.21638647e-04 1.32604502e-04
 2.03964007e-04 2.20336714e-04 3.58283528e-04 9.52844116e-05
 4.88024692e-04 1.83090977e-04 5.47272015e-04 2.16000475e-04
 2.35216425e-04 5.04005732e-04 2.35636947e-04 2.30969044e-04
 3.07318592e-04 2.88922693e-04 1.65759822e-04 3.27947531e-04
 1.20354167e-04]
0.0003180226954984769


Average R2 SCORE 

In [ ]:
from sklearn.metrics import r2_score
n1= []
n2 = []
for i in range(len(actualPG[0])):
  c1=r2_score(actualPG[i],predictionPG[i])
  n1.append(c1)
n1=np.array(n1)

print("For PG:")
print(n1)
print(n1.sum()/len(n1))
for i in range(len(actualAngles[0])):
  c2=r2_score(actualAngles[i],predictionAngles[i])
  n2.append(c2)
print("For Angles:")

n2=np.array(n2) 
print(n2)
print((n2.sum()/len(n2)))

For PG:
[0.99999926 0.99999928 0.99999948 0.99999926 0.99999931]
0.9999993176899231
For Angles:
[0.99999997 0.99999997 0.99999999 0.99999998 0.99999999 0.99999998
 1.         0.99999996 0.99999998 0.99999999 1.         1.
 1.         1.         0.99999999 1.         0.99999997 1.
 0.99999997 1.         0.99999999 0.99999998 0.99999999 0.99999999
 0.99999999 0.99999999 1.         0.99999999 1.        ]
0.9999999877241363


insert in Pg without slack

In [ ]:
 def GettingSlackPG(PG_predicted_ws, PD_test_unNormalized):
  import pandas as pd
  PG_predicted_ws = pd.DataFrame(PG_predicted_ws.copy())
  PD_test_unNormalized=pd.DataFrame(PD_test_unNormalized.copy())
  PD_test_unNormalized["Total D"] = PD_test_unNormalized.sum(axis=1)
  PG_predicted_ws["Total G"] = PG_predicted_ws.sum(axis=1)
  print(PD_test_unNormalized)
  print(PG_predicted_ws)
  #for column in PD_test_unNormalized[['Total D']]:
  #for column in PG_predicted_ws[['Total G']]:
  slack_generation=PD_test_unNormalized["Total D"] - PG_predicted_ws["Total G"]
  print(slack_generation)
  return slack_generation, PG_predicted_ws

slack_generation,PG_predicted_ws=GettingSlackPG(PG_predicted_ws, PD_test_unNormalized)



              0         1         2    3  ...   26        27         28     Total D
0     20.361337  2.243246  7.615190  0.0  ...  0.0  2.588622   9.919785  191.076614
1     21.780447  2.514907  7.671214  0.0  ...  0.0  2.558370   9.881597  183.311624
2     20.177372  2.518244  7.386829  0.0  ...  0.0  2.214616  10.407245  183.523142
3     22.339288  2.412296  7.323495  0.0  ...  0.0  2.276543   9.730261  190.535711
4     19.569104  2.475544  8.088795  0.0  ...  0.0  2.286665   9.752798  188.794310
...         ...       ...       ...  ...  ...  ...       ...        ...         ...
5995  23.433518  2.390882  7.928629  0.0  ...  0.0  2.408992   9.677816  185.764616
5996  22.880542  2.351072  8.137881  0.0  ...  0.0  2.419977  10.850851  193.076220
5997  22.465675  2.379904  7.166897  0.0  ...  0.0  2.605899   9.663107  191.059984
5998  22.855952  2.337595  7.195315  0.0  ...  0.0  2.521408  10.775091  187.038860
5999  20.225835  2.314832  7.246958  0.0  ...  0.0  2.502674  10.175829  185

In [ ]:
 def GettingSlackPG2(PG_predicted_ws, PD_test_unNormalized):
  import pandas as pd
  PG_predicted_ws = pd.DataFrame(PG_predicted_ws.copy())
  PD_test_unNormalized=pd.DataFrame(PD_test_unNormalized.copy())
  PD_test_unNormalized["Total D"] = PD_test_unNormalized.sum(axis=1)
  PG_predicted_ws["Total G"] = PG_predicted_ws.sum(axis=1)
  print(PD_test_unNormalized)
  print(PG_predicted_ws)
  #for column in PD_test_unNormalized[['Total D']]:
  #for column in PG_predicted_ws[['Total G']]:
  slack_generation=PD_test_unNormalized["Total D"] - PG_predicted_ws["Total G"]
  print(slack_generation)
  return slack_generation, PG_predicted_ws


slack_generation2,PG_actual_ws=GettingSlackPG2(PG_actual_ws, PD_test_unNormalized)

              0         1         2    3  ...   26        27         28     Total D
0     20.361337  2.243246  7.615190  0.0  ...  0.0  2.588622   9.919785  191.076614
1     21.780447  2.514907  7.671214  0.0  ...  0.0  2.558370   9.881597  183.311624
2     20.177372  2.518244  7.386829  0.0  ...  0.0  2.214616  10.407245  183.523142
3     22.339288  2.412296  7.323495  0.0  ...  0.0  2.276543   9.730261  190.535711
4     19.569104  2.475544  8.088795  0.0  ...  0.0  2.286665   9.752798  188.794310
...         ...       ...       ...  ...  ...  ...       ...        ...         ...
5995  23.433518  2.390882  7.928629  0.0  ...  0.0  2.408992   9.677816  185.764616
5996  22.880542  2.351072  8.137881  0.0  ...  0.0  2.419977  10.850851  193.076220
5997  22.465675  2.379904  7.166897  0.0  ...  0.0  2.605899   9.663107  191.059984
5998  22.855952  2.337595  7.195315  0.0  ...  0.0  2.521408  10.775091  187.038860
5999  20.225835  2.314832  7.246958  0.0  ...  0.0  2.502674  10.175829  185

In [ ]:
def AddingSlackToPG(PG_predicted_ws,slack_generation, slack_adapted,slack,ppc):
  #print(PG_predicted_ws)
  PG_predicted_ws.insert(slack_adapted,str(slack),slack_generation)
  #print(PG_predicted_ws)
  PG_predicted_ws = PG_predicted_ws.drop(["Total G"], axis=1)
  #print(PG_predicted_ws)
  columnnames= []
  for row in ppc["gen"]:
    columnnames.append(row[0])
  PredictedPG_WithSlack = pd.DataFrame(PG_predicted_ws.to_numpy(), columns=columnnames)
  #print(Predicted_WithSlack)
  return PredictedPG_WithSlack

PredictedPG_WithSlack=AddingSlackToPG(PG_predicted_ws,slack_generation,slack_adapted,slack,ppc)

print(PredictedPG_WithSlack)


           0.0        1.0        12.0       21.0       22.0       26.0
0     45.046066  58.567200  22.415165  33.013878  16.012819  16.021479
1     43.841989  57.194839  22.030704  30.130928  15.053440  15.059727
2     43.868173  57.236217  22.040413  30.211750  15.080570  15.086032
3     44.959585  58.471851  22.388163  32.812920  15.946755  15.956442
4     44.687742  58.164429  22.301970  32.169102  15.731070  15.740003
...         ...        ...        ...        ...        ...        ...
5995  44.216261  57.632317  22.151636  31.045935  15.356520  15.361944
5996  45.339877  58.921783  22.516069  33.764889  16.262693  16.270905
5997  45.043092  58.562363  22.414860  33.008244  16.010962  16.020479
5998  44.412883  57.861137  22.213428  31.518250  15.513917  15.519237
5999  44.211916  57.632767  22.150402  31.040812  15.354616  15.359110

[6000 rows x 6 columns]


In [ ]:
def AddingSlackToPG2(PG_predicted_ws,slack_generation, slack_adapted,slack,ppc):
  #print(PG_predicted_ws)
  PG_predicted_ws.insert(slack_adapted,str(slack),slack_generation)
  #print(PG_predicted_ws)
  PG_predicted_ws = PG_predicted_ws.drop(["Total G"], axis=1)
  #print(PG_predicted_ws)
  columnnames= []
  for row in ppc["gen"]:
    columnnames.append(row[0])
  PredictedPG_WithSlack = pd.DataFrame(PG_predicted_ws.to_numpy(), columns=columnnames)
  #print(Predicted_WithSlack)
  return PredictedPG_WithSlack

ActualPG_WithSlack=AddingSlackToPG2(PG_actual_ws,slack_generation2,slack_adapted,slack,ppc)
print(PredictedPG_WithSlack)
print(ActualPG_WithSlack)

           0.0        1.0        12.0       21.0       22.0       26.0
0     45.046066  58.567200  22.415165  33.013878  16.012819  16.021479
1     43.841989  57.194839  22.030704  30.130928  15.053440  15.059727
2     43.868173  57.236217  22.040413  30.211750  15.080570  15.086032
3     44.959585  58.471851  22.388163  32.812920  15.946755  15.956442
4     44.687742  58.164429  22.301970  32.169102  15.731070  15.740003
...         ...        ...        ...        ...        ...        ...
5995  44.216261  57.632317  22.151636  31.045935  15.356520  15.361944
5996  45.339877  58.921783  22.516069  33.764889  16.262693  16.270905
5997  45.043092  58.562363  22.414860  33.008244  16.010962  16.020479
5998  44.412883  57.861137  22.213428  31.518250  15.513917  15.519237
5999  44.211916  57.632767  22.150402  31.040812  15.354616  15.359110

[6000 rows x 6 columns]
           0.0        1.0        12.0       21.0       22.0       26.0
0     45.020355  58.594700  22.406515  33.022453  16

In [ ]:
def AddingSlacktoAngles(predictionAngles, slack_adapted, slack,ppc):
  predictionAngles = pd.DataFrame(predictionAngles)
  slackdf = pd.DataFrame(0, index=range(len(predictionAngles)), columns=[str(slack)])
  predictionAngles.insert(slack_adapted,str(slack),slackdf)
  columnnames= []
  for row in ppc["bus"]:
    columnnames.append(row[0])
  PredictedAngles_WithSlack = pd.DataFrame(predictionAngles.to_numpy(), columns=columnnames)
  return PredictedAngles_WithSlack 

PredictedAngles_WithSlack=AddingSlacktoAngles(angles_predicted,slack_adapted,slack,ppc)
ActualAngles_WithSlack=AddingSlacktoAngles(angles_actual,slack_adapted,slack,ppc)
print(PredictedAngles_WithSlack)
print(ActualAngles_WithSlack)

      0.0       1.0       2.0   ...      27.0       28.0       29.0
0      0.0 -0.881684 -2.111804  ... -4.760627 -10.575032 -11.428269
1      0.0 -0.861864 -2.043490  ... -4.547956 -10.274039 -11.125485
2      0.0 -0.853640 -2.072381  ... -4.614492 -10.480622 -11.412709
3      0.0 -0.894024 -2.063313  ... -4.684382 -10.322383 -11.178665
4      0.0 -0.861770 -2.135858  ... -4.737419 -10.542760 -11.400667
...    ...       ...       ...  ...       ...        ...        ...
5995   0.0 -0.882040 -2.020344  ... -4.536018 -10.160027 -11.001032
5996   0.0 -0.899807 -2.086400  ... -4.731382 -10.751744 -11.715387
5997   0.0 -0.893429 -2.074286  ... -4.697387 -10.424119 -11.249070
5998   0.0 -0.883915 -2.035812  ... -4.635184 -10.592493 -11.540691
5999   0.0 -0.863465 -2.078692  ... -4.677532 -10.485731 -11.372244

[6000 rows x 30 columns]
      0.0       1.0       2.0   ...      27.0       28.0       29.0
0      0.0 -0.880890 -2.111521  ... -4.760084 -10.574489 -11.427726
1      0.0 -0.861135 -

In [ ]:
def AddingSlacktoAngles2(predictionAngles, slack_adapted, slack,ppc):
  predictionAngles = pd.DataFrame(predictionAngles)
  slackdf = pd.DataFrame(0, index=range(len(predictionAngles)), columns=[str(slack)])
  predictionAngles.insert(slack_adapted,str(slack),slackdf)
  columnnames= []
  for row in ppc["bus"]:
    columnnames.append(row[0])
  PredictedAngles_WithSlack = pd.DataFrame(predictionAngles.to_numpy(), columns=columnnames)
  return PredictedAngles_WithSlack 

ActualAngles_WithSlack=AddingSlacktoAngles2(angles_actual,slack_adapted,slack,ppc)
print(ActualAngles_WithSlac)

ValueError: ignored

In [ ]:
def GenerationCost(PG,ppc):
  PGCost = PG.copy()
  PGCost = PGCost.to_numpy()
  for col in range(PGCost.shape[1]):
    PGCost[:,col] = ((PGCost[:,col])  * (PGCost[:,col]) * ppc["gencost"][col][4] )+ PGCost[:,col] *  ppc["gencost"][col][5] + ppc["gencost"][col][6]
  PGCost= pd.DataFrame(PGCost)
  PGCost["Total Cost"] = PGCost.sum(axis=1)
  PGCostAvg = PGCost["Total Cost"].mean()
  return PGCost, PGCostAvg

PGCost, PGCostAvg= GenerationCost(PredictedPG_WithSlack,ppc)
print("The cost of each prediction is")
print(PGCost)
print("The average cost of the predictions is:")
print(PGCostAvg)

The cost of each prediction is
               0           1          2  ...          4          5  Total Cost
0     130.675094  162.519645  79.806485  ...  54.448717  54.210582  582.794159
1     126.126380  157.337838  78.225912  ...  50.825472  50.835585  550.224166
2     126.224680  157.493111  78.265734  ...  50.927301  50.927691  551.097131
3     130.346457  162.157494  79.695233  ...  54.197741  53.981867  580.484694
4     129.315371  160.992014  79.340355  ...  53.379872  53.221224  573.096132
...          ...         ...        ...  ...        ...        ...         ...
5995  127.534077  158.982523  78.722283  ...  51.965127  51.894470  560.385042
5996  131.793843  163.869211  80.222543  ...  55.399960  55.088391  591.393069
5997  130.663788  162.501265  79.805228  ...  54.441657  54.207066  582.723757
5998  128.275850  159.845437  78.976196  ...  52.558791  52.446180  565.708207
5999  127.517702  158.984218  78.717214  ...  51.957954  51.884532  560.323181

[6000 rows x 7 colum

In [ ]:
PGCost2, PGCostAvg2= GenerationCost(ActualPG_WithSlack,ppc)
print("The cost of each actual PG is")
print(PGCost2)
print("The average cost of the actual PG is:")
print(PGCostAvg2)

The cost of each actual PG is
               0           1          2  ...          4          5  Total Cost
0     130.577357  162.624155  79.770843  ...  54.461911  54.192335  582.804206
1     126.038299  157.436661  78.189920  ...  50.830654  50.818413  550.231851
2     126.161205  157.577078  78.232890  ...  50.928958  50.910115  551.102829
3     130.259355  162.260667  79.660483  ...  54.207465  53.956809  580.495406
4     129.237302  161.092582  79.305405  ...  53.389807  53.199059  573.102770
...          ...         ...        ...  ...        ...        ...         ...
5995  127.465998  159.068266  78.688541  ...  51.972791  51.882579  560.388581
5996  131.755614  163.970716  80.179154  ...  55.404499  55.063668  591.391457
5997  130.567563  162.612976  79.767456  ...  54.454082  54.185089  582.733124
5998  128.209876  159.918442  78.947853  ...  52.567912  52.435984  565.713127
5999  127.457214  159.058291  78.685494  ...  51.965791  51.876072  560.326102

[6000 rows x 7 column

Checking for first PGi constraint violations

Checking for second Thetha constraint violation 